In [ ]:
import os
import shutil
from PIL import Image # Para verificar dimensões da imagem para os bboxes
import matplotlib.pyplot as plt # Para visualização de bboxes
import matplotlib.patches as patches # Para desenhar retângulos
import cv2 # Usado na seção de verificação opcional, pode ser necessário para ler imagens

In [ ]:
import os
import shutil

# --- Configurações de Caminho ---
FINAL_MERGED_IMAGE_DIR = "all_dataset_images" # Nome da pasta para juntar todas as imagens

# Certifica-se de que a pasta de destino final existe
if not os.path.exists(FINAL_MERGED_IMAGE_DIR):
    os.makedirs(FINAL_MERGED_IMAGE_DIR)
    print(f"Pasta de destino '{FINAL_MERGED_IMAGE_DIR}' criada.")

# Diretórios de onde as imagens serão COLETADAS
image_dirs = [
    'PART_1/PART_1/images',
    'PART_2/PART_2/images',
    'PART_3/PART_3/images'
]

# Extensões de arquivo de imagem que serão consideradas
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png')

# --- Processo de Junção de Imagens ---

print("\nIniciando a cópia das imagens...")
copied_image_count = 0

for source_dir in image_dirs:
    # Verifica se o diretório de origem existe antes de tentar listar
    if not os.path.exists(source_dir):
        print(f"Aviso: Diretório de origem '{source_dir}' não encontrado. Pulando...")
        continue

    print(f"Processando diretório: {source_dir}")
    for fname in os.listdir(source_dir):
        # Verifica se o arquivo é uma imagem pela extensão
        if fname.lower().endswith(IMAGE_EXTENSIONS):
            source_image_path = os.path.join(source_dir, fname)
            base_name, extension = os.path.splitext(fname)
            destination_image_name = f"{base_name}{extension}"
            destination_image_path = os.path.join(FINAL_MERGED_IMAGE_DIR, destination_image_name)

            try:
                shutil.copy2(source_image_path, destination_image_path)
                # print(f"Copiado: {source_image_path} -> {destination_image_path}")
                copied_image_count += 1
            except Exception as e:
                print(f"ERRO ao copiar '{source_image_path}': {e}")

print(f"\nProcesso concluído. Total de {copied_image_count} imagens copiadas para '{FINAL_MERGED_IMAGE_DIR}'.")

In [ ]:
import os
import shutil

# --- Configurações de Caminho ---
FINAL_MERGED_ANNOTATION = "all_dataset_annotations" # Nome da pasta para juntar todas as anotações

# Certifica-se de que a pasta de destino final existe
if not os.path.exists(FINAL_MERGED_ANNOTATION):
    os.makedirs(FINAL_MERGED_ANNOTATION)
    print(f"Pasta de destino '{FINAL_MERGED_ANNOTATION}' criada.")

# Diretórios de onde as anotações serão COLETADAS
ANNOTATIONs = ['PART_1/PART_1/6categories']

# Extensões de arquivo de imagem que serão consideradas
ANNOTATIONs_EXTENSION = ('.txt')

# --- Processo de Junção de anotações ---

print("\nIniciando a cópia das anotações...")
copied_image_count = 0

for source_dir in ANNOTATIONs:
    # Verifica se o diretório de origem existe antes de tentar listar
    if not os.path.exists(source_dir):
        print(f"Aviso: Diretório de origem '{source_dir}' não encontrado. Pulando...")
        continue

    print(f"Processando diretório: {source_dir}")
    for fname in os.listdir(source_dir):
        # Verifica se o arquivo é uma imagem pela extensão
        if fname.lower().endswith(ANNOTATIONs_EXTENSION):
            source_image_path = os.path.join(source_dir, fname)
            base_name, extension = os.path.splitext(fname)
            destination_image_name = f"{base_name}{extension}"
            destination_image_path = os.path.join(FINAL_MERGED_ANNOTATION, destination_image_name)

            try:
                shutil.copy2(source_image_path, destination_image_path)
                # print(f"Copiado: {source_image_path} -> {destination_image_path}")
                copied_image_count += 1
            except Exception as e:
                print(f"ERRO ao copiar '{source_image_path}': {e}")

print(f"\nProcesso concluído. Total de {copied_image_count} anotações copiadas para '{FINAL_MERGED_ANNOTATION}'.")

In [ ]:
import os

# Define o caminho para a pasta de anotações e o arquivo de nomes das classes
annotations_dir = "all_dataset_annotations"
names_file = "PART_1/PART_1/6categories.names"
# Sugiro um novo nome para o diretório de saída para não sobrescrever o anterior
output_dir = "cleaned_dataset_annotations_no_bouy" 

# Cria o diretório de saída se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1. Ler o arquivo de nomes das classes para mapear nomes para índices
class_names = []
with open(names_file, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

# 2. Identificar o índice da classe a ser REMOVIDA ("bouy")
bouy_idx = -1
try:
    bouy_idx = class_names.index("bouy")
except ValueError:
    print(f"Erro: A classe 'bouy' não foi encontrada em {names_file}.")
    print(f"Classes lidas: {class_names}")
    exit()

print(f"Índice da classe 'bouy' a ser removida: {bouy_idx}")
print(f"Todas as outras classes serão mantidas: {', '.join([name for i, name in enumerate(class_names) if i != bouy_idx])}")

# 3. Iterar sobre cada arquivo de anotação no diretório
for filename in os.listdir(annotations_dir):
    if filename.endswith(".txt"):
        input_filepath = os.path.join(annotations_dir, filename)
        output_filepath = os.path.join(output_dir, filename)
        
        kept_annotations = [] # Renomeado para maior clareza
        
        with open(input_filepath, 'r') as infile:
            for line in infile:
                parts = line.strip().split()
                if not parts:
                    continue
                
                try:
                    class_id = int(parts[0])
                    # Manter a linha APENAS SE a classe NÃO FOR 'bouy'
                    if class_id != bouy_idx:
                        kept_annotations.append(line)
                except ValueError:
                    print(f"Aviso: Linha inválida ignorada em {filename}: {line.strip()}")
                    continue
        
        # 4. Escrever as anotações filtradas em um novo arquivo
        if kept_annotations:
            with open(output_filepath, 'w') as outfile:
                for annotation in kept_annotations:
                    outfile.write(annotation)
            print(f"Arquivo filtrado salvo em: {output_filepath}")
        else:
            # Isso pode acontecer se um arquivo SÓ tinha anotações de 'bouy'
            print(f"Nenhuma anotação mantida para o arquivo: {filename} (após remover 'bouy'). Nenhum arquivo de saída gerado.")

print("\nLimpeza (remoção de 'bouy') concluída!")

In [ ]:
import os

# --- Configurações ---
# Diretório com TODAS as imagens (com base no seu notebook, parece ser este)
images_dir = "all_dataset_images" 

# Diretório com as anotações JÁ LIMPAS (o resultado do seu script anterior)
annotations_dir = "cleaned_dataset_annotations_no_bouy"

# --- Lógica do Script ---

# 1. Obter a lista de nomes base das anotações existentes
# Usamos um 'set' para uma busca muito mais rápida
annotation_basenames = set()
for ann_file in os.listdir(annotations_dir):
    if ann_file.endswith(".txt"):
        # Pega o nome do arquivo sem a extensão .txt (ex: 'img_001')
        basename = os.path.splitext(ann_file)[0]
        annotation_basenames.add(basename)

print(f"Encontradas {len(annotation_basenames)} anotações limpas em '{annotations_dir}'.")

# 2. Identificar imagens que não têm uma anotação correspondente
images_to_delete = []
for img_file in os.listdir(images_dir):
    # Pega o nome do arquivo de imagem sem a extensão (ex: 'img_001')
    img_basename = os.path.splitext(img_file)[0]
    
    # Se o nome base da imagem NÃO está na lista de nomes de anotações, ela deve ser apagada
    if img_basename not in annotation_basenames:
        images_to_delete.append(os.path.join(images_dir, img_file))

# 3. Revisão e Confirmação (PASSO DE SEGURANÇA)
if not images_to_delete:
    print("\nNenhuma imagem para apagar. Todas as imagens têm anotações correspondentes.")
else:
    print(f"\nAs seguintes {len(images_to_delete)} imagens serão APAGADAS pois não possuem anotações:")
    print(images_to_delete)
    # Mostra as primeiras 10 imagens para o usuário ter uma ideia
    for i, image_path in enumerate(images_to_delete[:10]):
        print(f" - {os.path.basename(image_path)}")
    if len(images_to_delete) > 10:
        print(f"   ... e mais {len(images_to_delete) - 10} outras.")

    # Pede confirmação ao usuário antes de apagar qualquer coisa
    # Mude para 's' se seu sistema for em português
    user_input = input("\nVocê tem certeza que deseja apagar esses arquivos? (s/n): ").lower()
    
    # 4. Apagar os arquivos se confirmado
    if user_input == 's' or user_input == 'y':
        deleted_count = 0
        for image_path in images_to_delete:
            try:
                os.remove(image_path)
                deleted_count += 1
            except OSError as e:
                print(f"Erro ao apagar o arquivo {image_path}: {e}")
        
        print(f"\nProcesso concluído. {deleted_count} imagens foram apagadas.")
    else:
        print("\nOperação cancelada pelo usuário. Nenhum arquivo foi apagado.")

In [1]:
import os

# --- Configurações de Caminho ---
input_dir = "cleaned_dataset_annotations_no_bouy"  # Pasta com anotações sem bouy
output_dir = "remapped_annotations"  # Nova pasta com classes remapeadas

# Cria o diretório de saída se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Pasta de destino '{output_dir}' criada.")

# --- Mapeamento de Classes ---
# Classes originais: human(0), wind-sup-board(1), boat(2), sailboat(4), kayak(5)
# Após remapeamento: human(0), boat(2)

class_mapping = {
    0: 0,  # human -> human (mantém)
    1: 0,  # wind-sup-board -> human  
    2: 2,  # boat -> boat (mantém)
    4: 2,  # sailboat -> boat
    5: 2   # kayak -> boat
}

print("Mapeamento de classes:")
print("  human (0) -> human (0)")
print("  wind-sup-board (1) -> human (0)")
print("  boat (2) -> boat (2)")
print("  sailboat (4) -> boat (2)")
print("  kayak (5) -> boat (2)")

# --- Processar arquivos ---
processed_files = 0
total_remapped_annotations = 0

for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, filename)
        
        remapped_annotations = []
        file_remapped_count = 0
        
        with open(input_filepath, 'r') as infile:
            for line in infile:
                parts = line.strip().split()
                if not parts:
                    continue
                
                try:
                    original_class_id = int(parts[0])
                    
                    # Verificar se a classe existe no mapeamento
                    if original_class_id in class_mapping:
                        new_class_id = class_mapping[original_class_id]
                        
                        # Se a classe mudou, contar como remapeada
                        if original_class_id != new_class_id:
                            file_remapped_count += 1
                            total_remapped_annotations += 1
                        
                        # Substituir o ID da classe
                        parts[0] = str(new_class_id)
                        
                        # Adicionar a linha processada
                        remapped_annotations.append(' '.join(parts) + '\n')
                    else:
                        print(f"Aviso: Classe {original_class_id} não encontrada no mapeamento em {filename}")
                        
                except ValueError:
                    print(f"Aviso: Linha inválida ignorada em {filename}: {line.strip()}")
                    continue
        
        # Escrever arquivo de saída
        if remapped_annotations:
            with open(output_filepath, 'w') as outfile:
                for annotation in remapped_annotations:
                    outfile.write(annotation)
            
            if file_remapped_count > 0:
                print(f"Arquivo {filename}: {file_remapped_count} anotações remapeadas")
        
        processed_files += 1

print(f"\nEtapa 1 concluída!")
print(f"  - {processed_files} arquivos processados")
print(f"  - {total_remapped_annotations} anotações remapeadas no total")
print(f"  - Arquivos salvos em: {output_dir}")
print(f"  - Agora você tem apenas classes: human (0) e boat (2)")

Pasta de destino 'remapped_annotations' criada.
Mapeamento de classes:
  human (0) -> human (0)
  wind-sup-board (1) -> human (0)
  boat (2) -> boat (2)
  sailboat (4) -> boat (2)
  kayak (5) -> boat (2)
Arquivo av_1381.txt: 6 anotações remapeadas
Arquivo av_249.txt: 2 anotações remapeadas
Arquivo a_101.txt: 2 anotações remapeadas
Arquivo a_1013.txt: 6 anotações remapeadas
Arquivo a_1014.txt: 6 anotações remapeadas
Arquivo a_1015.txt: 6 anotações remapeadas
Arquivo a_1016.txt: 6 anotações remapeadas
Arquivo a_1017.txt: 6 anotações remapeadas
Arquivo a_1018.txt: 6 anotações remapeadas
Arquivo a_1019.txt: 6 anotações remapeadas
Arquivo a_102.txt: 2 anotações remapeadas
Arquivo a_1020.txt: 6 anotações remapeadas
Arquivo a_1021.txt: 6 anotações remapeadas
Arquivo a_1022.txt: 6 anotações remapeadas
Arquivo a_1023.txt: 6 anotações remapeadas
Arquivo a_1024.txt: 6 anotações remapeadas
Arquivo a_1025.txt: 6 anotações remapeadas
Arquivo a_1026.txt: 6 anotações remapeadas
Arquivo a_1027.txt: 6 a

In [2]:
import os

# --- Configurações de Caminho ---
input_dir = "remapped_annotations"  # Pasta da etapa anterior
output_dir = "final_human_boat_dataset"  # Pasta final com human(0) e boat(1)

# Cria o diretório de saída se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Pasta de destino '{output_dir}' criada.")

# --- Mapeamento Final ---
# Da etapa anterior temos: human(0) e boat(2)
# Agora queremos: human(0) e boat(1)

final_mapping = {
    0: 0,  # human -> human (mantém 0)
    2: 1   # boat -> boat (muda para 1)
}

print("Mapeamento final:")
print("  human (0) -> human (0)")
print("  boat (2) -> boat (1)")

# --- Processar arquivos ---
processed_files = 0
total_annotations = 0
human_count = 0
boat_count = 0

for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, filename)
        
        final_annotations = []
        
        with open(input_filepath, 'r') as infile:
            for line in infile:
                parts = line.strip().split()
                if not parts:
                    continue
                
                try:
                    class_id = int(parts[0])
                    
                    # Aplicar mapeamento final
                    if class_id in final_mapping:
                        new_class_id = final_mapping[class_id]
                        parts[0] = str(new_class_id)
                        
                        # Contar as classes
                        if new_class_id == 0:
                            human_count += 1
                        elif new_class_id == 1:
                            boat_count += 1
                        
                        total_annotations += 1
                        final_annotations.append(' '.join(parts) + '\n')
                    else:
                        print(f"Aviso: Classe {class_id} inesperada em {filename}")
                        
                except ValueError:
                    print(f"Aviso: Linha inválida ignorada em {filename}: {line.strip()}")
                    continue
        
        # Escrever arquivo final
        if final_annotations:
            with open(output_filepath, 'w') as outfile:
                for annotation in final_annotations:
                    outfile.write(annotation)
        
        processed_files += 1

# Criar arquivo .names para o dataset final
names_file = os.path.join(output_dir, "classes.names")
with open(names_file, 'w') as f:
    f.write("human\n")
    f.write("boat\n")

print(f"\nEtapa 2 concluída!")
print(f"  - {processed_files} arquivos processados")
print(f"  - {total_annotations} anotações totais")
print(f"  - {human_count} anotações de human (classe 0)")
print(f"  - {boat_count} anotações de boat (classe 1)")
print(f"  - Arquivos salvos em: {output_dir}")
print(f"  - Arquivo de classes criado: {names_file}")
print(f"\nDataset final pronto com apenas 2 classes: human(0) e boat(1)!")

Pasta de destino 'final_human_boat_dataset' criada.
Mapeamento final:
  human (0) -> human (0)
  boat (2) -> boat (1)

Etapa 2 concluída!
  - 2912 arquivos processados
  - 39404 anotações totais
  - 37096 anotações de human (classe 0)
  - 2308 anotações de boat (classe 1)
  - Arquivos salvos em: final_human_boat_dataset
  - Arquivo de classes criado: final_human_boat_dataset\classes.names

Dataset final pronto com apenas 2 classes: human(0) e boat(1)!


In [ ]:
import os
import shutil
import random
from pathlib import Path

def organize_yolo_dataset(images_dir, annotations_dir, output_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Organiza dataset YOLO em pastas train/val/test
    
    Args:
        images_dir: caminho para pasta com imagens .jpg
        annotations_dir: caminho para pasta com anotações .txt
        output_dir: caminho onde criar as pastas organizadas
        train_ratio: proporção para treino (0.7 = 70%)
        val_ratio: proporção para validação (0.2 = 20%)
        test_ratio: proporção para teste (0.1 = 10%)
    """
    
    # Criar estrutura de pastas
    output_path = Path(output_dir)
    for split in ['train', 'val', 'test']:
        (output_path / split / 'images').mkdir(parents=True, exist_ok=True)
        (output_path / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Listar todas as imagens
    images_path = Path(images_dir)
    image_files = list(images_path.glob('*.jpg'))
    
    # Verificar se existem anotações correspondentes
    annotations_path = Path(annotations_dir)
    valid_pairs = []
    
    for img_file in image_files:
        txt_file = annotations_path / f"{img_file.stem}.txt"
        if txt_file.exists():
            valid_pairs.append((img_file, txt_file))
        else:
            print(f"Aviso: Anotação não encontrada para {img_file.name}")
    
    print(f"Total de pares válidos (imagem + anotação): {len(valid_pairs)}")
    
    # Embaralhar os dados
    random.shuffle(valid_pairs)
    
    # Calcular divisões
    total = len(valid_pairs)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    
    splits = {
        'train': valid_pairs[:train_end],
        'val': valid_pairs[train_end:val_end],
        'test': valid_pairs[val_end:]
    }
    
    # Copiar arquivos para as respectivas pastas
    for split_name, pairs in splits.items():
        print(f"\nCopiando {len(pairs)} arquivos para {split_name}...")
        
        for img_file, txt_file in pairs:
            # Copiar imagem
            dst_img = output_path / split_name / 'images' / img_file.name
            shutil.copy2(img_file, dst_img)
            
            # Copiar anotação
            dst_txt = output_path / split_name / 'labels' / txt_file.name
            shutil.copy2(txt_file, dst_txt)
    
    print(f"\nDataset organizado com sucesso!")
    print(f"Train: {len(splits['train'])} amostras")
    print(f"Val: {len(splits['val'])} amostras") 
    print(f"Test: {len(splits['test'])} amostras")


In [3]:
# Substitua pelos seus caminhos reais
organize_yolo_dataset(
    images_dir='final_human_boat_dataset',
    annotations_dir='remapped_annotations', 
    output_dir='dataset_yolo'
)

Total de pares válidos (imagem + anotação): 0

Copiando 0 arquivos para train...

Copiando 0 arquivos para val...

Copiando 0 arquivos para test...

Dataset organizado com sucesso!
Train: 0 amostras
Val: 0 amostras
Test: 0 amostras
